# Flats analysis
#### Continuation of flats dynamic webscraping notebook

### For each of considered areas the aim is to:
1. Find all flats on the last floor and has at least 3 rooms.
2. Reporting those flats will continue for a period of time, so:
3. Flats are to be differentiated weather they have already been reported or not.
<br>   - 'start_date' and 'start_index' indicate the starting date of a current analysis. Flats that were collected earlier are to be separated from those "newest" ones.
<br>   - 'last_date' and 'last_index' indicate the "freshiest" data.
<br>   - settings.json file is used to store a date of the last run of the analysis that will be used to establish 'start_date' and 'start_index', for each considered area.

In [1]:
import json
import numpy as np
import os
import pandas as pd
import re
from datetime import date
from itertools import groupby
from IPython.core.debugger import set_trace

In [1]:
#########################################################
#
##        Uncomment the one to be proceeded
#
########################################################

#where = 'wlochy'
where = 'ochota'

In [3]:
# To play a sound whenever exception is thrown
# source: https://gist.github.com/krassowski/c6373c155e264bedd7581129a30c7657
from IPython.display import Audio, display

wave2 = np.sin(3*np.pi*400*np.arange(10000*0.5)/10000)

def play_sound(self, etype, value, tb, tb_offset=None):
    self.showtraceback((etype, value, tb), tb_offset=tb_offset)
    display(Audio(wave2, rate=10000, autoplay=True))
    
#get_ipython().set_custom_exc((ZeroDivisionError,), play_sound)  # any Error(s) can be listed in a tuple after ','
get_ipython().set_custom_exc((Exception,), play_sound)  # the sound will be played at any exception thrown

In [4]:
with open("settings.json", "r") as read_file:
    json_settings = json.load(read_file)

In [5]:
json_settings['last_run_date_ochota']

'2023-09-12'

In [6]:
json_settings['last_run_date_wlochy']

'2023-09-12'

#### List of csv files to be read and csv files indexes

1. Assumed csv header:
<br>Offert_no, URL,Price,Price_unit,Rent,Rent_unit,Area,Area_unit,Rooms_no,Floor,Max_floor
2. Order of files:
<br>oldest to newest.
3. <b>--> csv_files_tuples list<b>
4. start_date and start_index - which file to start the analysis with
5. last_date and last_index - markers of the last csv file

In [8]:
# The date value of the newest csv file
last_run_date = ''
if where == 'ochota':
    last_run_date = str(json_settings['last_run_date_ochota'])
if where == 'wlochy':
    last_run_date = str(json_settings['last_run_date_wlochy'])
last_date = '0000-00-00'
last_index = 0

# The date to start analysis with
start_date = '1111-11-11'
start_index = 0
is_start = False

csv_files_tuples = []

where_plus = where + '_flats_rent'
flats_rent_path = os.path.join('.', where_plus)
print(flats_rent_path)
if not os.path.exists(flats_rent_path):
    # create dir
    os.makedirs(flats_rent_path)
else:
    #read files in that dir to a csv_files_tuples list
    flat_rent_lst = os.listdir(flats_rent_path)
    #print(flat_rent_lst)
    idx = 1
    #csv_files_tuples_1 = []
    for ls in flat_rent_lst:
        if ls.endswith('.csv'):
            csv_files_tuples.append((idx, ls))
            if not is_start and (date.fromisoformat(ls[0:10]) > date.fromisoformat(last_run_date)):
                is_start = True
                start_date = ls[0:10]
                start_index = idx
            idx += 1
    print(f'\n {csv_files_tuples[0]} ... {csv_files_tuples[-1]}')
    last_index = csv_files_tuples[-1][0]
    last_date = csv_files_tuples[-1][1][0:10]
    print(f'last_index and last_date: {last_index}, {last_date}')
    print(f'start_index and date: {start_index}, {start_date}')

if start_index == 0:  # what means repeat the analysis on the last date
    start_date = last_date
    start_index = last_index
print(f'start_index and date, corrected: {start_index}, {start_date}')

.\ochota_flats_rent
['.ipynb_checkpoints', '2023-07-20--otodom_ochota_flats.csv', '2023-07-25--otodom_ochota_flats.csv', '2023-07-26--otodom_ochota_flats.csv', '2023-07-27--otodom_ochota_flats.csv', '2023-07-28--otodom_ochota_flats.csv', '2023-08-03--otodom_ochota_flats.csv', '2023-08-07--otodom_ochota_flats.csv', '2023-08-08--otodom_ochota_flats.csv', '2023-08-09--otodom_ochota_flats.csv', '2023-08-11--otodom_ochota_flats.csv', '2023-08-14--otodom_ochota_flats.csv', '2023-08-16--otodom_ochota_flats.csv', '2023-08-17--otodom_ochota_flats.csv', '2023-08-18--otodom_ochota_flats.csv', '2023-08-21--otodom_ochota_flats.csv', '2023-08-22--otodom_ochota_flats.csv', '2023-08-23--otodom_ochota_flats.csv', '2023-08-24--otodom_ochota_flats.csv', '2023-08-25--otodom_ochota_flats.csv', '2023-08-28--otodom_ochota_flats.csv', '2023-08-30--otodom_ochota_flats.csv', '2023-08-31--otodom_ochota_flats.csv', '2023-09-01--otodom_ochota_flats.csv', '2023-09-04--otodom_ochota_flats.csv', '2023-09-05--otodom_o

#### Functions to read .csv file

1. Extracts date from the csv file name.
2. Reads csv file do a pandas DataFrame df, setting dtypes and what values treat as NaN.
3. Inserts at the left-most side of df new columns 'Csv_file_no' and 'Date' with extracted file index on the csv_files_tuples list and date value.
4. Return: df

In [10]:
def read_csv_with_date_from_tuples(csv_file_tuple):
    regex_date_pattern = re.compile('\d{4}-\d{2}-\d{2}')
    date = re.search(regex_date_pattern, csv_file_tuple[1]).group(0)
    dtypes = {'Offert_no': object,
              'URL': object,
              'Price': 'Float64', 
              'Price_unit': object, 
              'Rent': 'Float64',
              'Rent_unit': object,
              'Area': 'Float64',
              'Area_unit': object,
              'Rooms_no': 'Int64',
              'Floor': 'Int64',
              'Max_floor': 'Int64'}
    where_plus = where + '_flats_rent'
    csv_path = os.path.join('.', where_plus, csv_file_tuple[1])
    #df = pd.read_csv(csv_file_tuple[1], dtype=dtypes, na_values=['', 'null'])
    #set_trace()
    df = pd.read_csv(csv_path, dtype=dtypes, na_values=['', 'null'])
    df.insert(0, 'Date', date)
    df.insert(0, 'Csv_file_no', csv_file_tuple[0])
    #print(df.head())
    return df

#### Reading all csv files

As listed in csv_files above.

In [11]:
# define exception
class Error(Exception):
   """Base class for other exceptions"""
   pass

class NoFilesTuplesError(Error):
    """Raised when list of files tuples is empty.

    Attributes:
        message -- explanation of the error
    """

    def __init__(self, message):
        self.message = message

# reading csv files
if csv_files_tuples:
    df_orig = pd.concat(map(read_csv_with_date_from_tuples, csv_files_tuples)).reset_index(drop=True)
else:
    raise NoFilesTuplesError('No data to proceed with. Terminating the analysis.')
    


In [12]:
df_orig.head()

,Csv_file_no,Date,Offert_no,URL,Price,Price_unit,Rent,Rent_unit,Area,Area_unit,Rooms_no,Floor,Max_floor
0,1,2023-07-20,64401406.0,https://www.otodom.pl/pl/oferta/wynajme-eksklu...,9000.0,zł,1200.0,zł/miesiąc,90.0,m²,3,3,3
1,1,2023-07-20,64405843.0,https://www.otodom.pl/pl/oferta/wygodne-2-poko...,3500.0,zł,<NA>,zł/miesiąc,48.0,m²,2,1,4
2,1,2023-07-20,NaN,https://www.otodom.pl/pl/oferta/mieszk-2-pokoj...,3100.0,zł,900.0,zł/miesiąc,47.0,m²,2,10,10
3,1,2023-07-20,NaN,https://www.otodom.pl/pl/oferta/zielen-cisza-w...,2500.0,zł,565.0,zł/miesiąc,35.0,m²,1,5,6
4,1,2023-07-20,NaN,https://www.otodom.pl/pl/oferta/mieszkanie-2-p...,3000.0,zł,800.0,zł/miesiąc,38.0,m²,2,3,8


In [13]:
df_orig.tail()

,Csv_file_no,Date,Offert_no,URL,Price,Price_unit,Rent,Rent_unit,Area,Area_unit,Rooms_no,Floor,Max_floor
6645,30,2023-09-13,64580945,https://www.otodom.pl/pl/oferta/premium-82m2-4...,9000.0,zł,700.0,zł/miesiąc,82.0,m²,4,0,5
6646,30,2023-09-13,64540155,https://www.otodom.pl/pl/oferta/dwupokojowe-mi...,3600.0,zł,<NA>,zł/miesiąc,40.0,m²,2,0,4
6647,30,2023-09-13,64550801,https://www.otodom.pl/pl/oferta/klimatyczne-3-...,5400.0,zł,<NA>,zł/miesiąc,57.0,m²,3,2,8
6648,30,2023-09-13,64572165,https://www.otodom.pl/pl/oferta/nowe-mieszkani...,4300.0,zł,700.0,zł/miesiąc,44.0,m²,2,7,7
6649,30,2023-09-13,64556230,https://www.otodom.pl/pl/oferta/mieszkanie-war...,4500.0,zł,800.0,zł/miesiąc,51.0,m²,3,4,10


#### De-duplicate rows based on columns: URL and Price.

- Row are assumed to be duplicating if their URLs and Prices values are identical.
- Those that has different Price values are to be kept for the purpose of price in time analysis. 
- If any rows are duplicated, keep the last one, assuming that last one is more up-to-date.

In [14]:
print('Before de-duplication: ', len(df_orig.index))
df_URL_deduplicated = df_orig.drop_duplicates(subset=['URL', 'Price'], keep='last', ignore_index=True)
print('After de-duplication: ', len(df_URL_deduplicated.index))

Before de-duplication:  6650
After de-duplication:  946


### Flats to concider in current analysis:

1. from the start date till the last one
2. what means excluding all that has ever appeared before start date first
3. then de-duplicating based on URL and Price
<br><b> --> df_newest<b>

In [15]:
################################################################
#
#     Uncomment and set manually, if needed
#
################################################################

#start_index = 25
#start_date = '2023-09-05'

In [16]:
# !!! flats that has > 3 rooms, are on last or >10 floor and their price has changed in files later than start_index

# first dictionary out of df_orig_csv_sorted slownik, for csv_file_no == start_index-1 but without csv_file_no
if start_index > 1:
    before_dict = pd.Series(df_orig[(df_orig['Csv_file_no'] == (start_index - 1)) & (df_orig['Rooms_no'] >= 3) & ((df_orig['Floor'] == -10) | (df_orig['Floor'] == df_orig['Max_floor']))][['URL', 'Price']].sort_values(by=['URL']).Price.values,index=df_orig[(df_orig['Csv_file_no'] == (start_index - 1)) & (df_orig['Rooms_no'] >= 3) & ((df_orig['Floor'] == -10) | (df_orig['Floor'] == df_orig['Max_floor']))][['URL', 'Price']].sort_values(by=['URL']).URL).to_dict()
    #print(before_dict)
    
# second, analogic dictionary for csv_file_no == last_index, what means current file
current_dict = pd.Series(df_orig[(df_orig['Csv_file_no'] == last_index) & (df_orig['Rooms_no'] >= 3) & ((df_orig['Floor'] == -10) | (df_orig['Floor'] == df_orig['Max_floor']))][['URL', 'Price']].sort_values(by=['URL']).Price.values,index=df_orig[(df_orig['Csv_file_no'] == last_index) & (df_orig['Rooms_no'] >= 3) & ((df_orig['Floor'] == -10) | (df_orig['Floor'] == df_orig['Max_floor']))][['URL', 'Price']].sort_values(by=['URL']).URL).to_dict()
#print(current_dict)

changed_price_URLs_list = []

# compare both dictionaries, if the second one contains URL that is also in the first on and their price values are different (has changed) - add it to changed_price_URLs list
for key, value in current_dict.items():
    if key in before_dict and before_dict[key] != value:
        changed_price_URLs_list.append(key)
print(f'changed_price_URLs_list: {changed_price_URLs_list}')

{'https://www.otodom.pl/pl/oferta/3-niezalezne-pokoje-dickensa-bezposredno-wawa-ID4n0Pm': 3000.0, 'https://www.otodom.pl/pl/oferta/3-pokojowe-mieszkanie-na-ochocie-ID4n3oL': 6500.0, 'https://www.otodom.pl/pl/oferta/apartament-na-szczesliwicach-ID4mTMl': 6000.0, 'https://www.otodom.pl/pl/oferta/apartament-na-wynajem-w-kameralnej-kamienicy-ID4md25': 16800.0, 'https://www.otodom.pl/pl/oferta/duze-mieszkanie-przy-parku-szczesliwickim-ID4jFXC': 10000.0, 'https://www.otodom.pl/pl/oferta/duze-mieszkanie-z-dwoma-tarasami-na-szczesliwicach-ID4iGCh': 8000.0, 'https://www.otodom.pl/pl/oferta/dwupoziomowy-apartament-z-tarasami-na-ochocie-ID4ck3F': 25000.0, 'https://www.otodom.pl/pl/oferta/ekskluzywne-z-sauna-silka-3-pokoje-ID4mrCE': 8999.0, 'https://www.otodom.pl/pl/oferta/ekskluzywny-penthouse-przy-parku-szczesliwickim-ID43zQO': 30000.0, 'https://www.otodom.pl/pl/oferta/filtry-kolonia-staszica-dwa-pokoje-i-gabinet-ID4mA2m': 4900.0, 'https://www.otodom.pl/pl/oferta/luksusowy-apartament-na-ochocie-

In [17]:
# URL values that are not to be concidered in the analysis: are in file with csv file no. < start_index unless their price has changed
exclude_URL_values = set(df_orig[(df_orig['Csv_file_no'] < start_index) & ~df_orig['URL'].isin(changed_price_URLs_list)]['URL'].unique())

In [18]:
df_newest = df_orig[~df_orig['URL'].isin(exclude_URL_values) & (df_orig['Csv_file_no'] == last_index)].drop_duplicates(subset=['URL', 'Price'], keep='last', ignore_index=True)

In [19]:
print()
print(len(df_newest.index), 'flats to be concidered in current analysis\n')
print()
df_newest.head()


19 flats to be concidered in current analysis




,Csv_file_no,Date,Offert_no,URL,Price,Price_unit,Rent,Rent_unit,Area,Area_unit,Rooms_no,Floor,Max_floor
0,30,2023-09-13,64606705,https://www.otodom.pl/pl/oferta/ladne-2-pokoje...,2900.0,zł,600.0,zł/miesiąc,30.0,m²,2,2,4
1,30,2023-09-13,64605632,https://www.otodom.pl/pl/oferta/do-wynajecia-3...,4700.0,zł,870.0,zł/miesiąc,75.0,m²,3,4,6
2,30,2023-09-13,64604397,https://www.otodom.pl/pl/oferta/3-pokojowe-mie...,5500.0,zł,1500.0,zł/miesiąc,91.3,m²,3,3,4
3,30,2023-09-13,64603840,https://www.otodom.pl/pl/oferta/2-pokojowe-mie...,3050.0,zł,650.0,zł/miesiąc,41.3,m²,2,1,12
4,30,2023-09-13,64603166,https://www.otodom.pl/pl/oferta/mieszkanie-war...,3500.0,zł,<NA>,zł/miesiąc,40.0,m²,1,3,3


#### Function to remove consecutive duplicates from a list

Input: [1,1,1,1,1,1,2,3,4,4,5,1,2,2,2]
<br>Result: [1, 2, 3, 4, 5, 1, 2]

In [20]:
def deduplicate_consecutive(lst):
    return [key for key, _group in groupby(lst)]

#### Flats data printing function

In [21]:
def print_flat_data(flats_list, number):
    empty_str = '   (Dziś tu pusto...)'
    num = number
    if flats_list:
        #for num, flat in enumerate(newest_last_below_mean):
        for flat in flats_list:
            prev_prices = ''
            price_ls = df_orig[(df_orig['URL'] == flat)][['Csv_file_no', 'Price']].sort_values(by=['Csv_file_no'], ascending=True).Price.to_list()
            if price_ls:
                #remove consecutive duplicates
                price_ls = deduplicate_consecutive(price_ls)
                for price in price_ls:
                    prev_prices = prev_prices + ' ' + f'{price:.2f}'
            #print(num + 1, flat, '\n', prev_prices)
            print(num, flat, '\n', prev_prices)
            num += 1
    else:
        print(f'{empty_str}')
    return num

## THE REPORT

In [22]:
where_polished = ''
if where == 'ochota':
    where_polished = 'Ochota'
if where == 'wlochy':
    where_polished = 'Włochy'

print(f'{where_polished}, mieszkania na wynajem, raporcik\n\n')
print(f'{where_polished}, WYNAJEM, {date.today()}.\n')
mean_price_over_area = (df_URL_deduplicated[(df_URL_deduplicated['Csv_file_no'] == last_index)]['Price'] / df_URL_deduplicated[(df_URL_deduplicated['Csv_file_no'] == last_index)]['Area']).mean()
print(f'Średnia cena najmu za m^2 (bez czynszu): {mean_price_over_area:.2f}zł')
print()

print(f'{where_polished}, czynsze:')
#how much are rent values in the neighbourhood (== in Ochota or in Włochy)
min_rent = df_URL_deduplicated[(df_URL_deduplicated['Csv_file_no'] == last_index) & (df_URL_deduplicated['Rent'] >= 100)]['Rent'].min()
min_index = df_URL_deduplicated[(df_URL_deduplicated['Csv_file_no'] == last_index) & (df_URL_deduplicated['Rent'] == min_rent)].index.to_numpy()
min_area = df_URL_deduplicated.loc[min_index[0]]['Area']
min_URL = df_URL_deduplicated.loc[min_index[0]]['URL']
#set_trace()
print(f'- najniższy: {min_rent:.2f}zł, za mieszkanie o powierzchni {min_area:.2f} m^2: {min_URL}')
max_rent = df_URL_deduplicated[(df_URL_deduplicated['Csv_file_no'] == last_index)]['Rent'].max()
#set_trace()
max_index = df_URL_deduplicated[(df_URL_deduplicated['Csv_file_no'] == last_index) & (df_URL_deduplicated['Rent'] == max_rent)].index.to_numpy()
max_area = df_URL_deduplicated.loc[max_index[0]]['Area']
max_URL = df_URL_deduplicated.loc[max_index[0]]['URL']
print(f'- najwyższy: {max_rent:.2f}zł, za mieszkanie o powierzchni {max_area:.2f} m^2: {max_URL}')
mean_rent = df_URL_deduplicated[df_URL_deduplicated['Csv_file_no'] == last_index]['Rent'].astype(float).mean()
print(f'- średni: {mean_rent:.2f}zł,')
median_rent = df_URL_deduplicated[df_URL_deduplicated['Csv_file_no'] == last_index].Rent.astype(float).median()
print(f'- mediana: {median_rent:.2f}zł,')
print()

num = 1
empty_str = '   (Dziś tu pusto...)'
print('\nMieszkania i historia ich cen, w formacie:')
print('[Nr kolejny] [URL]')
print('[Historia cen, od najstarszej (w zł)]')
print('\nMieszkania z min. 3 pokojami na ostatnim piętrze, najnowsze:')
print('- o cenie najmu za m^2 poniżej średniej:')
# show only last occurences if duplicated and all non-duplicated as well:
newest_last_below_mean = df_newest[((~df_newest.duplicated(subset=['URL', 'Price'], keep='last') & df_newest.duplicated(subset=['URL', 'Price'], keep=False)) | (~df_newest.duplicated(subset=['URL', 'Price'], keep=False))) & 
                                   (df_newest['Rooms_no'] >= 3) & (df_newest['Floor'] == df_newest['Max_floor']) & 
                                   ((df_newest['Price'] / df_newest['Area']) < mean_price_over_area)]['URL'].sort_values().to_list()
num = print_flat_data(newest_last_below_mean, num)

print('\n- pozostałe:')
newest_last_above_mean = df_newest[((~df_newest.duplicated(subset=['URL', 'Price'], keep='last') & df_newest.duplicated(subset=['URL', 'Price'], keep=False)) | (~df_newest.duplicated(subset=['URL', 'Price'], keep=False))) & 
                                    (df_newest['Rooms_no'] >= 3) & (df_newest['Floor'] == df_newest['Max_floor']) & 
                                    ((df_newest['Price'] / df_newest['Area']) >= mean_price_over_area)]['URL'].sort_values().to_list()
num = print_flat_data(newest_last_above_mean, num)

print('\nMieszkania z min. 3 pokojami, na piętrze > 10, najnowsze:')
print('- o cenie najmu za m^2 poniżej średniej:')
newest_10more_below_mean = df_newest[((~df_newest.duplicated(subset=['URL', 'Price'], keep='last') & df_newest.duplicated(subset=['URL', 'Price'], keep=False)) | (~df_newest.duplicated(subset=['URL', 'Price'], keep=False))) &
                                     (df_newest['Rooms_no'] >= 3) & (df_newest['Floor'] == -10) & 
                                     ((df_newest['Price'] / df_newest['Area']) < mean_price_over_area)]['URL'].sort_values().to_list()
num = print_flat_data(newest_10more_below_mean, num)
    
print('\n- pozostałe:')
newest_10more_above_mean = df_newest[((~df_newest.duplicated(subset=['URL', 'Price'], keep='last') & df_newest.duplicated(subset=['URL', 'Price'], keep=False)) | (~df_newest.duplicated(subset=['URL', 'Price'], keep=False))) &
                                     (df_newest['Rooms_no'] >= 3) & (df_newest['Floor'] == -10) & 
                                     ((df_newest['Price'] / df_newest['Area']) >= mean_price_over_area)]['URL'].sort_values().to_list()
num = print_flat_data(newest_10more_above_mean, num)

print(f'\nMieszkania z min. 3 pokojami na ostatnim piętrze, starsze niż {start_date}:')
print('- o cenie najmu za m^2 poniżej średniej:')
older_last_below_mean = df_URL_deduplicated[(df_URL_deduplicated['Csv_file_no'] == last_index) & 
                                            (df_URL_deduplicated['URL'].isin(exclude_URL_values)) &
                                            (df_URL_deduplicated['Rooms_no'] >= 3) & (df_URL_deduplicated['Floor'] == df_URL_deduplicated['Max_floor']) & 
                                            ((df_URL_deduplicated['Price'] / df_URL_deduplicated['Area']) < mean_price_over_area)]['URL'].sort_values().to_list()
num = print_flat_data(older_last_below_mean, num)

print('\n- pozostałe:')
older_last_above_mean = df_URL_deduplicated[(df_URL_deduplicated['Csv_file_no'] == last_index) & 
                                            (df_URL_deduplicated['URL'].isin(exclude_URL_values)) &
                                            (df_URL_deduplicated['Rooms_no'] >= 3) & (df_URL_deduplicated['Floor'] == df_URL_deduplicated['Max_floor']) & 
                                            ((df_URL_deduplicated['Price'] / df_URL_deduplicated['Area']) >= mean_price_over_area)]['URL'].sort_values().to_list()
num = print_flat_data(older_last_above_mean, num)

print(f'\nMieszkania z min. 3 pokojami, na piętrze > 10, starsze niż {start_date}:')
print('- o cenie najmu za m^2 poniżej średniej:')
older_10more_below_mean = df_URL_deduplicated[(df_URL_deduplicated['Csv_file_no'] == last_index) & 
                                              (df_URL_deduplicated['URL'].isin(exclude_URL_values)) &
                                              (df_URL_deduplicated['Rooms_no'] >= 3) & (df_URL_deduplicated['Floor'] == -10) &
                                              ((df_URL_deduplicated['Price'] / df_URL_deduplicated['Area']) < mean_price_over_area)]['URL'].sort_values().to_list()
num = print_flat_data(older_10more_below_mean, num)

print('\n- pozostałe:')
older_10more_above_mean = df_URL_deduplicated[(df_URL_deduplicated['Csv_file_no'] == last_index) & 
                                              (df_URL_deduplicated['URL'].isin(exclude_URL_values)) &
                                              (df_URL_deduplicated['Rooms_no'] >= 3) & (df_URL_deduplicated['Floor'] == -10) &
                                              ((df_URL_deduplicated['Price'] / df_URL_deduplicated['Area']) >= mean_price_over_area)]['URL'].sort_values().to_list()
num = print_flat_data(older_10more_above_mean, num)

# Store in settings.json info about 'last_run_date' as date.today()
if where == 'ochota':
    json_settings['last_run_date_ochota'] = str(date.today())
if where == 'wlochy':
    json_settings['last_run_date_wlochy'] = str(date.today())
with open("settings.json", "w") as write_file:
    json.dump(json_settings, write_file)

Ochota, mieszkania na wynajem, raporcik


Ochota, WYNAJEM, 2023-09-13.

Średnia cena najmu za m^2 (bez czynszu): 83.09zł

Ochota, czynsze:
- najniższy: 200.00zł, za mieszkanie o powierzchni 20.00 m^2: https://www.otodom.pl/pl/oferta/fresh-studio-city-centre-warsaw-ID4mQew
- najwyższy: 1900.00zł, za mieszkanie o powierzchni 110.00 m^2: https://www.otodom.pl/pl/oferta/apartament-na-szczesliwicach-ID4mTMl
- średni: 761.83zł,
- mediana: 700.00zł,


Mieszkania i historia ich cen, w formacie:
[Nr kolejny] [URL]
[Historia cen, od najstarszej (w zł)]

Mieszkania z min. 3 pokojami na ostatnim piętrze, najnowsze:
- o cenie najmu za m^2 poniżej średniej:
   (Dziś tu pusto...)

- pozostałe:
   (Dziś tu pusto...)

Mieszkania z min. 3 pokojami, na piętrze > 10, najnowsze:
- o cenie najmu za m^2 poniżej średniej:
   (Dziś tu pusto...)

- pozostałe:
   (Dziś tu pusto...)

Mieszkania z min. 3 pokojami na ostatnim piętrze, starsze niż 2023-09-13:
- o cenie najmu za m^2 poniżej średniej:
1 https://www.ot

In [59]:
%debug

None
> c:\users\maria-a\appdata\local\temp\ipykernel_22436\2430994583.py(24)<module>()



ipdb>  max_rent


nan


ipdb>  q
